In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def min_max_norm(val, min_val, max_val, new_min, new_max):
  return (val - min_val) * (new_max - new_min) / (max_val - min_val) + new_min

In [ ]:
class Chromosome:
  def __init__(self, length, array=None): #if array is None it should be initialized with random binary vector
    if array is None:
      array = [random.choice([0, 1]) for _ in range(length)]
    self.array = array
    self.length = length


  def decode(self, lower_bound, upper_bound, aoi):
    pass

  def mutation(self, probability):
    if random.random() < probability:
      index = random.randint(0, self.length-1)
      self.array[index] = ~self.array[index] & 1 #bitwise NOT

  def crossover(self, other):
    index = random.randint(0, self.length-1)
    output_array_1 = np.concatenate((self.array[0:index], other.array[index:]), axis=0)
    output_array_2 = np.concatenate((other.array[0:index], self.array[index:]), axis=0)
    return Chromosome(self.length, output_array_1), Chromosome(self.length, output_array_2)

In [ ]:
class GeneticAlgorithm:
  def __init__(self, chromosome_length, obj_func_num_args, objective_function, aoi, population_size=1000,
               tournament_size=2, mutation_probability=0.05, crossover_probability=0.8, num_steps=30):
    assert chromosome_length % obj_func_num_args == 0, "Number of bits for each argument should be equal"
    self.chromosome_lengths = chromosome_length
    self.obj_func_num_args = obj_func_num_args
    self.bits_per_arg = int(chromosome_length / obj_func_num_args)
    self.objective_function = objective_function
    self.aoi = aoi
    self.tournament_size = tournament_size
    self.mutation_probability = mutation_probability
    self.crossover_probability = crossover_probability
    self.num_steps = num_steps

  def eval_objective_func(self, chromosome):
    pass

  def tournament_selection(self):
    pass

  def reproduce(self, parents):
    if random.random() < self.crossover_probability:
      return parents[0].crossover(parents[1])
    else:
      return parents


  def plot_func(self, trace):
    X = np.arange(-2, 3, 0.05)
    Y = np.arange(-4, 2, 0.05)
    X, Y = np.meshgrid(X, Y)
    Z = 1.5 - np.exp(-X ** (2) - Y ** (2)) - 0.5 * np.exp(-(X - 1) ** (2) - (Y + 2) ** (2))
    plt.figure()
    plt.contour(X, Y, Z, 10)
    cmaps = [[ii / len(trace), 0, 0] for ii in range(len(trace))]
    plt.scatter([x[0] for x in trace], [x[1] for x in trace], c=cmaps)
    plt.show()

  def run(self):
    pass